In [ ]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# On avge, 252 trading days a year
NUM_TRADING_DAYS = 252

# Generate random w (diff. portfolios)
NUM_PORTFOLIOS = 10000

# Our portfolio stocks
stocks = ['AAPL', 'NVDA', 'MAR', 'LMT', 'BA']

# Historical data - define START, END dates
start_date = '2010-01-01'
end_date = '2023-10-01'

In [ ]:
merged = pd.read_excel("merged.xlsx", index_col=0)

In [ ]:
import numpy as np
import pandas as pd

stock_dict = {}
for stock in stocks:
    new_df = merged.filter(regex=f'{stock}_')
    stock_dict[f"{stock}"] = new_df

In [ ]:
AAPL_test= stock_dict["AAPL"]

In [ ]:
training_list = ["AAPL_Daily_Return", "AAPL_Adj Close", "AAPL_Volume", "AAPL_Sentiment_score"]
AAPL_df = AAPL_test.loc[:, training_list]
AAPL_test = AAPL_test.loc[:, training_list].values
# AAPL_test = AAPL_test.loc[:, training_list]
print(AAPL_test)
AAPL_df


In [ ]:
# sc_dailyReturn = MinMaxScaler(feature_range=(0,1))
# sc_adjClose = MinMaxScaler(feature_range=(0,1))
# sc_Volume = MinMaxScaler(feature_range=(0,1))
# sc_sentimentScore = MinMaxScaler(feature_range=(0,1))
# dailyReturn = sc_dailyReturn.fit_transform(AAPL_df["AAPL_Daily_Return"])
# adjClose = sc_adjClose.fit_transform(AAPL_df["AAPL_Adj Close"])
# volume = sc_Volume.fit_transform(AAPL_df["AAPL_Volume"])
# sentimentScore = sc_sentimentScore.fit_transform(AAPL_df["AAPL_Sentiment_score"])
# print(dailyReturn)

sc = MinMaxScaler(feature_range=(0,1))
AAPL_test = sc.fit_transform(AAPL_test)

# print(AAPL_test)

X = []
y = []

# print(len(AAPL_test.index)) #3474
length = len(AAPL_test)
# print(length)
sequence_length = 60

for i in range(length - sequence_length):
    X.append(AAPL_test[i:i + sequence_length])
    y.append(AAPL_test[i + sequence_length])

X, y = np.array(X), np.array(y)

In [ ]:
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# creation of validation set from train data
# X_val, X_train, y_val, y_train = train_test_split(X_train, y_train, test_size=0.8, random_state=42)

# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 4))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 4))

print(y_test.shape)
print(X_test.shape)

# y_test


In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import regularizers

In [ ]:
model = Sequential()

model.add(LSTM(128, input_shape=(sequence_length, 4)))
# model.add(LSTM(64, return_sequences=False))
# model.add(Dense(25))

# Dense(4) because we are predicting 4 features
model.add(Dense(4))

# model.add(LSTM(128, return_sequences=True, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape= (sequence_length, 4)))
# model.add(LSTM(64, return_sequences=False, kernel_regularizer=regularizers.l2(0.001)))
# model.add(Dense(25, kernel_regularizer=regularizers.l2(0.001)))
# model.add(Dense(1, kernel_regularizer=regularizers.l2(0.001)))

model.compile(optimizer='adam', loss='mean_squared_error')
# history = model.fit(X_train,y_train, validation_data=(X_val, y_val), epochs=50, batch_size=100)
history = model.fit(X_train,y_train, epochs=50, batch_size=100)

training_loss = history.history['loss']
# validation_loss = history.history['val_loss']
epochs = range(1, len(training_loss) + 1)

# Plot the loss function
plt.figure(figsize=(10, 5))
plt.plot(epochs, training_loss, label='Training Loss')
# plt.plot(epochs, validation_loss, label='Validation Loss')
plt.title('Loss Function for LSTM Model')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
y_pred = model.predict(X_test)

# only interested in daily returns
y_test_slice = y_test[:,0] #MinMaxScaled
y_pred_slice = y_pred[:,0] #MinMaxScaled

# AAPL_test = sc.inverse_transform(AAPL_test)
y_pred = sc.inverse_transform(y_pred)

print(np.sqrt(mean_squared_error(y_test_slice, y_pred_slice)))

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have y_test_original_scale and y_pred_original_scale

# Create a range of indices for plotting
indices = range(len(y_test_slice))

# Plot the actual stock prices (in blue)
plt.plot(indices, y_test_slice, label="Actual Stock Returns", color='blue')

# Plot the predicted stock prices (in red)
plt.plot(indices, y_pred_slice, label="Predicted Stock Returns", color='red')

# Add labels and legend
plt.xlabel("Date")
plt.ylabel("Stock Returns")
plt.title("Actual vs. Predicted Stock Returns")
plt.legend()

# Show the plot
plt.show()

In [ ]:
!pip install scikeras

In [ ]:
import tensorflow as tf
from scikeras.wrappers import KerasRegressor


In [ ]:
def create_lstm_model(units=50, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units, kernel_regularizer=regularizers.l2(learning_rate), input_shape=(sequence_length, 4)))
    model.add(Dense(4))
    model.compile(optimizer='adam', loss='mean_squared_error')
    print("Creating model in function...")
    return model

    
# verbose = 0
# Create the LSTM model
lstm_model = KerasRegressor(model=create_lstm_model, learning_rate=0.001, units=50)
# lstm_model = KerasRegressor(model=create_lstm_model)
print("Created model!")




In [ ]:
# Specify hyperparameter values to search
param_grid = {
    'units': [50, 100],
    'learning_rate': [0.001, 0.01],
    # from the loss curve earlier, early stopping
    # optimal number of epoch < 10, else overfit
    'epochs': [5, 10],
    'batch_size': [30, 50]
}

# Perform grid search
grid_search = GridSearchCV(estimator=lstm_model, param_grid=param_grid, cv=2)
print("Initialised GridSearchCV")
grid_search.fit(X_train, y_train)  # X_train and y_train are your training data

# Get the best hyperparameters
best_units = grid_search.best_params_['units']
best_learning_rate = grid_search.best_params_['learning_rate']
best_epochs = grid_search.best_params_['epochs']
best_batch_size = grid_search.best_params_['batch_size']

print(best_units, best_learning_rate, best_epochs, best_batch_size)

In [ ]:
# Train the final model with the best hyperparameters

final_model = create_lstm_model(units=best_units, learning_rate=best_learning_rate)
model.fit(X_train,y_train, epochs=best_epochs, batch_size=best_batch_size)

In [ ]:
y_pred = model.predict(X_test)

# only interested in daily returns
y_test_slice = y_test[:,0] #MinMaxScaled
y_pred_slice = y_pred[:,0] #MinMaxScaled

# AAPL_test = sc.inverse_transform(AAPL_test)
y_pred = sc.inverse_transform(y_pred)

print(np.sqrt(mean_squared_error(y_test_slice, y_pred_slice)))

y_pred

In [ ]:
import matplotlib.pyplot as plt

# Create a range of indices for plotting
indices = range(len(y_test_slice))

# Plot the actual stock prices (in blue)
plt.plot(indices, y_test_slice, label="Actual Stock Returns", color='blue')

# Plot the predicted stock prices (in red)
plt.plot(indices, y_pred_slice, label="Predicted Stock Returns", color='red')

# Add labels and legend
plt.xlabel("Time Step")
plt.ylabel("Stock Return")
plt.title("Actual vs. Predicted Stock Return")
plt.legend()

# Show the plot
plt.show()